# Imports

In [6]:
import pandas as pd
import json
from urllib.request import Request, urlopen, HTTPError
import re
import datetime
from datetime import datetime as dt
from bs4 import BeautifulSoup as soup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tweepy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import nltk
from collections import Counter
import time
from tqdm import tqdm
import sys
import requests
from apis import get_TrackICOAPI


consumer_key = os.getenv("TWITTER_PUBLIC_API")
consumer_secret = os.getenv("TWITTER_SECRET_KEY")
nltk.download('vader_lexicon')

addl_stopwords = [',','`', '', 'rt', 'http', 'https', 'RT', 'BTC', 'bitcoin', 'ETH', 'LTC', 'XRP', 'co', 'crypto', 'blockchain', 'cryptocurrency', 'cripto', 'litecoin']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cscat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Begin work on ICO Projects

In [2]:
ico = ICO_data()

NameError: name 'ICO_data' is not defined

In [6]:
ico_df = ico.preprocess_data()

In [7]:
ico_df

,Name,Description,Price(USD),Start,End,ROI(Pct),Timezone,Duration
Ticker,,,,,,,,
REP,Augur,Augur is a decentralized prediction market,11.16,2015-08-17 00:00:00,2015-10-01 00:00:00,18.2409,UTC+0,45 days 00:00:00
LSK,Lisk,It is a cryptocurrency and decentralized appli...,0.78,2016-02-22 00:00:00,2016-03-21 00:00:00,9.3831,UTC+0,28 days 00:00:00
DGD,Digix DAO,A Decentralized Autonomous Organization focuse...,11.85,2016-03-30 12:00:00,2016-03-30 12:00:00,2.6574,UTC+0,0 days 00:00:00
WAVES,Waves,Waves helps to make the launching and coordina...,0.77,2016-04-12 13:00:00,2016-05-31 13:00:00,3.1065,UTC+0,49 days 00:00:00
STRAT,Stratis,Stratis was developed to help organisations de...,0.30,2016-06-21 00:00:00,2016-07-26 00:00:00,39.7913,UTC+0,35 days 00:00:00
XTO,Tao Network,Tao Network posits itself as a digital asset m...,0.03,2016-07-16 00:00:00,2016-08-15 00:00:00,1.7510,UTC+0,30 days 00:00:00
INCNT,Incent,Incent is a blockchain based loyalty program f...,0.17,2016-09-01 00:00:00,2016-09-01 00:00:00,2.5213,UTC+0,0 days 00:00:00
NEO,NEO,NEO is a China based smart contract Blockchain...,12.17,2016-08-08 00:00:00,2016-09-07 00:00:00,75.5492,UTC+0,30 days 00:00:00
BLOCKPAY,BlockPay,This is a retail platform that allows multicry...,0.06,2016-08-15 00:00:00,2016-09-15 00:00:00,-0.5169,UTC+0,31 days 00:00:00


In [5]:
!git status

On branch Christian
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Christian_working_notebook.ipynb
	__init__.py
	__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [18]:
#ICO Watch List API Wrapper- Can be called with 'live', 'upcoming', and 'finished' on the end for their respective lists, otherwise calls all ICO's
#No API key needed, 1sec limit per call

class ICO_data():
    
    # Initiates the object (self) and allows variables to be set for use in any of the classes funtions
    def __init__(self):
        self.url =' https://api.icowatchlist.com/public/v1/'
        
    def get_json(self):
        '''
        Sends HTTP Request to provided url and returns a json (dictionary) object.

        Arguements: 'url' - Requires a full http address including any applicable API keys.
        '''
        request = Request(self.url, headers={'User-Agent': 'Python'})
        response = urlopen(request)
        raw_data = response.read()
        json_data = json.loads(raw_data)
        return json_data

    def get_ico_df(self):
        '''
        Performs the 'get_json()' funtion and converts it into a Pandas DataFrame
        '''
        json_data = self.get_json()
        ico_list = json_data['ico']['finished']
        df = pd.DataFrame(ico_list)
        return df


     
    
    def preprocess_data(self):
        '''
        Performs the 'get_df' function and removes erronus columns, converts time to DateTime objects and 
        converts the numbers to floats
        '''
        df = self.get_ico_df()
        
        df.drop(columns=['icowatchlist_url', 'image', 'website_link'], inplace = True)
        reordered_columns = ['Name', 'Description', 'Price(USD)', 'Start', 'End', 'ROI(Pct)', 'Timezone']
        df.rename(columns={'all_time_roi': 'ROI(Pct)',
                           'coin_symbol': 'Ticker',
                          'description': 'Description',
                           'end_time': 'End',
                           'name': 'Name',
                           'price_usd': 'Price(USD)',
                          'start_time': 'Start',
                           'timezone': 'Timezone',
                          }, inplace=True)
        df.set_index('Ticker', inplace = True)
        df=df.reindex(columns=reordered_columns)
        #Convert the strings to datetime objects
        df['Start'] = df['Start'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
        df['End'] = df['End'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
        df['Price(USD)'] = df['Price(USD)'].replace("NA",'0')
        df['ROI(Pct)'] = df['ROI(Pct)'].replace('NA','0%')
        #Split the price values that are over 1,000 at the ','
        df['Price(USD)'] = df['Price(USD)'].apply(lambda x: re.split(',', x))
        #Split the roi values at their ',', and '%'
        df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: re.split('[, %]', x))
        #Merge the strings back that are over 1,000 and turn them into floats
        try:
            df['Price(USD)'] = df['Price(USD)'].apply(lambda x: float(x[0] + x[1])) 
        #Convert the remaining strings back to floats    
        except:
            df['Price(USD)'] = df['Price(USD)'].apply(lambda x: float(x[0]))
        
        try:
            df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: float(x[0] + x[1])/100) 
        except:
            df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: float(x[0])/100)
        #Create a duration column from the start and end dates
        df["Duration"] = df['End'] - df['Start']
        return df

  

In [41]:
  
def check_cmc(project_name):
    '''
    Check Coin Market Cap for the project, if it exists capture the current price, market cap, volume, circulating supply, total supply, ATH, ATL
    '''
    cmc_base_url = 'https://coinmarketcap.com/currencies/'
    headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
    request=Request(cmc_base_url + project_name, headers=headers) 
    response =urlopen(request)
    soup = BeautifulSoup(response, 'html.parser')
        
        

# Test ICO functions

In [5]:
#Instantiate the object
ico = ICO_data()

In [43]:
#Call the funtion and create the dateframe
clean_df = ico.preprocess_data()
clean_df.head()

,Name,Description,Price(USD),Start,End,ROI(Pct),Timezone,Duration
Ticker,,,,,,,,
REP,Augur,Augur is a decentralized prediction market,11.63,2015-08-17 00:00:00,2015-10-01 00:00:00,19.0495,UTC+0,45 days
LSK,Lisk,It is a cryptocurrency and decentralized appli...,0.75,2016-02-22 00:00:00,2016-03-21 00:00:00,9.1001,UTC+0,28 days
DGD,Digix DAO,A Decentralized Autonomous Organization focuse...,12.34,2016-03-30 12:00:00,2016-03-30 12:00:00,2.8086,UTC+0,0 days
WAVES,Waves,Waves helps to make the launching and coordina...,0.79,2016-04-12 13:00:00,2016-05-31 13:00:00,3.1867,UTC+0,49 days
STRAT,Stratis,Stratis was developed to help organisations de...,0.31,2016-06-21 00:00:00,2016-07-26 00:00:00,40.8281,UTC+0,35 days


# Begin work on Coin Projects

In [17]:
def tokenizer(text):
    """Tokenizes text."""
    addl_stopwords = ['ann', '']
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    sw = set(stopwords.words('english') + addl_stopwords)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    clean_text = [word for word in text if word not in sw]
    return clean_text



class Coin_data():
    
    def __init__(self):
        self.btctalk_ann_url = 'https://bitcointalk.org/index.php?board=159.0'
        self.cmc_base_url = 'https://coinmarketcap.com/currencies/'
        self.cmc_coin_url = 'https://coinmarketcap.com/all/views/all/'

    def get_cmc_coins(self):
        headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
        request=Request(self.cmc_coin_url, headers=headers) 
        response =urlopen(request)
        soup = BeautifulSoup(response, 'html.parser')
        coin_list = soup.findAll('a' , {'class':"currency-name-container link-secondary"})
        coins = []
        for coin in coin_list:
            coins.append(coin.text)
        return coins
        
        
    def get_new_projects(self):
        '''
        Gets a list of all new projects from Bitcointalk.org that are listed on CoinMarketCap
        '''
        #Get list of coins on Coin Market Cap
        print("Getting a list of all coins on Coin Market Cap")
        coin_list = self.get_cmc_coins()
        coin_list = [coin.lower() for coin in coin_list]
        
        #Prepare BS4 to scrape bitcointalk.org announcement page
        headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
        request=Request(self.btctalk_ann_url, headers=headers) 
        response =urlopen(request)
        soup = BeautifulSoup(response, 'html.parser')
        
        #Create a list of all the post on the announcement page
        links = soup.findAll('a')
        
        #Capture the url for each post
        links = [url.get('href') for url in links if 'ANN' in url.text]
        
        #For loop over each url saving the content of each page to a dict key
        print('Looping over each url saving the content of each page to a dict key')
        coins = {}
        count = 0
        for url in links:
            count += 1
            headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
            request=Request(url, headers=headers) 
            response =urlopen(request)
            soup = BeautifulSoup(response, 'html.parser')
            coins[count] = soup
            
        #For loop over each dict key (post html page) and captive the title, as well as search the body for key words
        print('Looping over each dict key (post html page) and captive the title, as well as search the body for key words')
        count = 1
        flags = ['gaurenteed', 'profit', 'government', 'approval', 'massive', 'mega', 'rich', 'money']
        name, rating = [],[]
        for i in range(len(coins)):
            count += 1
            scam_meter = 0

            for flag in flags:
                if flag in coins[i+1].text.lower():
                    scam_meter += 1
            name += [coins[i+1].title.text]
            rating += [scam_meter]
            
        #Create a dataframe to store the title and scam rating for each post
        df = pd.DataFrame({
            'Title':name,
            'Scam_Rating':rating,
        })
        return df
#commented out so the function will still run and return a df
    '''        
        #Extract the project name from the title
        print('Verifing if coin is on Coin Market Cap and removing those that are not')
        words, names=[], []
        for i in range(len(df)):
            words += tokenizer(df['Title'][i])
            for word in words:
                if word in coin_list:
                    cmc = 0
#                    names.append(word)
                else:
                    cmc = 1
                names.append(cmc)
#                    names.append('NA')
        print(names)           
#        df['Name'] = names
#        df = df.dropna()
'''                
        
        
        #Extract the ticker from the title
        
        
        #Extract a start date from the body of each post
        
        
        #Extract an end date from the body of each post
        
        
        #Capture the offering price for each coin
        
        
        #Identify the Algo for each project 
        


# Test the Coin functions

In [45]:
#Instantiate the object
coin = Coin_data()

In [46]:
#Call the funtion and create the dateframe
df = coin.get_new_projects()
df.head()

Getting a list of all coins on Coin Market Cap


NameError: name 'BeautifulSoup' is not defined

In [47]:
#Tokenize each Title and check if any words match the coin_names[] if so add that name to the dataframe otherwise set the name to 'NA'
print('Verifing if coin is on Coin Market Cap and removing those that are not')
words={}
for i in range(len(df)):
    words[i]=tokenizer(df['Title'][i])
words

Verifing if coin is on Coin Market Cap and removing those that are not


KeyError: 'Title'

In [ ]:
for words in words[6]:
    print(words)

In [ ]:
for i in range(len(words)):
    for words in words[i]:
        print(words)

### Code from Azerpas (github)

In [4]:
class ico():
	def __init__(self):
		self.hpages = 3
		self.url = "https://icobench.com/icos?filterSort=rating-desc&page="
		self.s = requests.session()
		self.book = []
		self.r = None

	def scrape(self):
		log("Scraping ICO data...")
		for i in range(1,self.hpages+1):
			req = self.s.get(self.url+str(i))
			soup = BeautifulSoup.BeautifulSoup(req.text)

			for i in soup.findAll("div",{"class":"rate color5"}):
				current_ico = (i.parent.parent)
				if not "ico_data" in str(current_ico):
					continue
				for z in current_ico.findAll("div"):
					if "Start" in z.text and "End" in z.text:
						pass
					if "Start" in z.text:
						pattern = re.compile("Start:(.*)")
						start_date = re.findall(pattern,z.text)
					if "End" in z.text:
						pattern = re.compile("End:(.*)")
						end_date = re.findall(pattern,z.text)
				for z in current_ico.findAll("a"):
					if "href" in str(z):
						pattern = re.compile('href="(.*)">')
						link = re.findall(pattern,str(z))
				title = current_ico.find("a",{"class":"name"})
				pattern = re.compile("""href="/ico/(.*)">""")
				title = re.findall(pattern,str(title))
				title = title[0].replace('-',' ').title()

				for z in current_ico.findAll("div",{"class":"content"}):
					for p in z.findAll("p"):
						description = p.text
				try:
					sep = "KYC"
					description = description.split(sep,1)[0]
				except Exception as e:
					pass

				rating = float(i.text)

				start_date = start_date[0]
				end_date = end_date[0]
				link = "https://icobench.com" + link[0]
				self.book.append({'title':title,'start':start_date,'end':end_date,'description':description,'link':link,'rating':rating})

		maxi = self.book[0]['rating']
		self.r = {'weekWinner':self.book[0]['title'],'rate':maxi}
		for i in range(0,len(self.book)):
			if self.book[i]['rating'] > maxi:
				maxi = self.book[i]['rating']
				self.r['rate'] = maxi
				self.r['weekWinner'] = self.book[i]['title']
		log("Scraped successfully")


	def syntax(self):
		message = "📈 Meilleure ICO de ces 4 derniers jours: " + self.r['weekWinner'] + " avec un score de: " + str(self.r['rate']) + "/5\n\n"
		cutting = 0
		for i in range(0,len(self.book)):
			cutting += 1
			message += "\nNom: {}\nDescription: {}\nDébut: {}\nFin: {}\nLien: {}\nNote: {}\n----------------\n".format(self.book[i]['title'],self.book[i]['description'],self.book[i]['start'],self.book[i]['end'],self.book[i]['link'],self.book[i]['rating'])
			if (cutting % 3) == 0:
				message += "+++"
		# adapted to ICObench syntax
		return message

### End

In [3]:
ico = ICO_data()

In [50]:
ico_df = ico.preprocess_data()

In [51]:
ico_df.iloc[0]

Name                                                Augur
Description    Augur is a decentralized prediction market
Price(USD)                                          11.63
Start                                 2015-08-17 00:00:00
End                                   2015-10-01 00:00:00
ROI(Pct)                                          19.0495
Timezone                                            UTC+0
Duration                                 45 days 00:00:00
Name: REP, dtype: object

In [7]:
def tokenizer(text):
    """Tokenizes text."""
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    sw = set(stopwords.words('english') + addl_stopwords)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    clean_text = [word for word in text if word not in sw]
    return clean_text

def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)


# Functions for Twitter


def get_tweets_list(topic_of_tweet, num_of_tweets):
    '''
    Returns a dataframe of the most recent 'N' tweets from Twitter tokenized and counted.
    
    Arguements: `topic_of_tweet` : str; what hashtag is being searched 
                'num_of_tweets' : int; how many tweet do you want returned
    '''
    text,time, word_list, word_count=[],[],[],[]
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth)
    for tweet in tweepy.Cursor(api.search, q=topic_of_tweet, tweet_mode='extended').items(num_of_tweets):
        text.append(tweet.full_text),
        time.append(tweet.created_at)
    tweets_df = pd.DataFrame({'Tweet':text}, index=time)
    [word_list.append(tokenizer(text)) for text in tweets_df.Tweet]
    tweets_df['Tokens'] = word_list
    [word_count.append(token_count(token)) for token in tweets_df.Tokens]
    tweets_df['Word_Count'] = word_count
    
    return tweets_df

def twitter_sent_analysis(tweet_df):    
    tweet_sentiments, comp, pos, neg, neu = [],[],[],[],[]
    analyzer = SentimentIntensityAnalyzer()
    for tweet in tweet_df.Tweet:
        sentiment = analyzer.polarity_scores(tweet),
        comp.append(sentiment[0]["compound"]),
        pos.append(sentiment[0]["pos"]),
        neg.append(sentiment[0]["neg"]),
        neu.append(sentiment[0]["neu"]),
  
    tweet_df['Compound'] = comp
    tweet_df['Positive'] = pos
    tweet_df['Negative'] = neg
    tweet_df['Neutral'] = neu

    return tweet_df

def count(df):
    '''
    Takes a DataFrame with a "compund" column and returns a basic count of positive, neutral, and negative sentiment in a dict format
    '''
    positive_count, negative_count, neutral_count = 0,0,0
    for i in df['Compound']:
        if i >= 0.05:
            positive_count += 1
        elif i <= -0.05:
            negative_count += 1
        else:
            neutral_count += 1
    count={
        'Positive Tweets': positive_count,
        'Neutral Tweets': neutral_count,
       'Negavtive Tweets': negative_count
    }
    return count

def get_twitter_scores(topic_of_tweet, num_of_tweets):
    df = get_tweets_list(topic_of_tweet, num_of_tweets)
    df = twitter_sent_analysis(df)
    
    return df

In [53]:
aug_twittwer_df = get_twitter_df(ico_df.index[1], 10)

NameError: name 'get_twitter_df' is not defined

In [8]:
def twitter_score(df):    
    scores = []
    for ico in df.Name:
        search_term = '#' + str(ico)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, 20)
        score = {ico :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{ico} scored")
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")
    
    return scores
    

In [ ]:
test_df = ico_df[0:5]

In [ ]:
test_df

In [ ]:
x = twitter_score(test_df)

In [ ]:
b=pd.DataFrame()
b.head()

In [ ]:
for item in x:
    t=pd.DataFrame.from_dict(item).T
    b = pd.concat([b,t], sort=True)
    b.head()

In [ ]:
b

In [ ]:
x ={
    'Compuond' : aug_twittwer_df['Compound'].mean(),
    'Positive' : aug_twittwer_df['Positive'].mean(),
    'Negative' : aug_twittwer_df['Negative'].mean(),
    'Neutral' : aug_twittwer_df['Neutral'].mean(),
}

In [ ]:
aug_twittwer_df.append(x, ignore_index=True)

In [ ]:
test_df = ico_df.iloc[0:3]

In [ ]:
test_df

In [54]:
test_df['Compound'] = ''
test_df['Positive'] = ''
test_df['Negative'] = ''
test_df['Neutral'] = ''

NameError: name 'test_df' is not defined

In [ ]:
test_df

In [ ]:
test_df.Compound[1] = aug_twittwer_df.Compound.mean()
test_df.Positive[1]= aug_twittwer_df.Positive.mean()
test_df.Negative[1]= aug_twittwer_df.Negative.mean()
test_df.Neutral[1] = aug_twittwer_df.Neutral.mean()

In [ ]:
test_df

In [ ]:
test_df.Negative[1]

In [9]:
def twitter_df_score(df, N): 
    '''
    Scores an entire Dataframe of coins based on the last 'N' tweets.  Returns a dataframe of scores with a 
    '''
    scores = []
    for name in df.Name:
        search_term = '#' + str(name)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, N)
        score = {name :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{name} scored")
        for i in progressbar(range(10), "Waiting for Twitter Rate Limit: ", 40):
            time.sleep(0.6) # any calculation you need
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")
    
    for item in scores:
        df1=pd.DataFrame.from_dict(item).T
        df2 = pd.concat([df1,df2], sort = True)
#    ndf = pd.concat([df, df2])

    return df2

In [113]:
twitter_scores = twitter_df_score(ico_df, 20)

Searching and Scoring #Augur
Augur scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Lisk
Lisk scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Digix DAO
Digix DAO scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Waves
Waves scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Stratis
Stratis scored


KeyboardInterrupt: 

In [10]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [35]:
for i in progressbar(range(10), "Computing: ", 40):
    time.sleep(.6) # any calculation you need

Computing: [########################################] 10/10


In [69]:
df = pd.read_csv('./df.csv')

In [70]:
df.dropna()

,Unnamed: 0,Name,Compound,Description,Duration,End,Negative,Neutral,Positive,Price(USD),ROI(Pct),Start,Ticker,Timezone


In [71]:
df_both = pd.read_csv('./df.csv')

In [72]:
df1 = df_both[:744]

In [73]:
df2 = df_both[743:]

In [74]:
df1 = df1.sort_values('Name')

In [75]:
df2 = df2.sort_values('Name')

In [76]:
df1.tail()

,Unnamed: 0,Name,Compound,Description,Duration,End,Negative,Neutral,Positive,Price(USD),ROI(Pct),Start,Ticker,Timezone
255,i,intimate.io,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619,i,jNet1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,i,kevin,0.00000,NaN,NaN,NaN,0.0000,1.0000,0.000,NaN,NaN,NaN,NaN,NaN
481,i,tZERO,-0.08695,NaN,NaN,NaN,0.0468,0.9322,0.021,NaN,NaN,NaN,NaN,NaN
724,i,vDice.io,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df2.tail()

,Unnamed: 0,Name,Compound,Description,Duration,End,Negative,Neutral,Positive,Price(USD),ROI(Pct),Start,Ticker,Timezone
1230,n,intimate.io,NaN,Intimate is a cryptocurrency facilitating paym...,12 days 00:00:00.000000000,2018-05-08 09:00:00,NaN,NaN,NaN,0.0,0.0000,2018-04-26 09:00:00,ITM,UTC+0
866,n,jNet1,NaN,jNET1 is a platform that enables B2B and B2C j...,29 days 00:00:00.000000000,2017-09-30 00:00:00,NaN,NaN,NaN,0.0,-0.9999,2017-09-01 00:00:00,JNT,UTC+3
968,n,kevin,NaN,kevin is an innovative online banking service ...,66 days 00:00:00.000000000,2017-12-22 00:00:00,NaN,NaN,NaN,0.0,0.0000,2017-10-17 00:00:00,KVT,UTC+0
1004,n,tZERO,NaN,The tZERO platform integrates cryptographicall...,31 days 00:00:00.000000000,2018-01-18 19:32:00,NaN,NaN,NaN,0.0,0.0000,2017-12-18 19:32:00,TZRO,UTC+0
761,n,vDice.io,NaN,Betting Game based on the Ethereum Blockchain,30 days 02:00:00.000000000,2016-12-15 02:00:00,NaN,NaN,NaN,0.0,0.0000,2016-11-15 00:00:00,VSLICE,UTC+0


In [78]:
df3 = pd.merge(df1, df2, on='Name')

In [79]:
df3

,Unnamed: 0_x,Name,Compound_x,Description_x,Duration_x,End_x,Negative_x,Neutral_x,Positive_x,Price(USD)_x,...,Duration_y,End_y,Negative_y,Neutral_y,Positive_y,Price(USD)_y,ROI(Pct)_y,Start_y,Ticker_y,Timezone_y
0,i,0x,0.242940,NaN,NaN,NaN,0.004850,0.904600,0.090550,NaN,...,5 days 00:00:00.000000000,2017-08-20 03:00:00,NaN,NaN,NaN,0.30,5.2644,2017-08-15 03:00:00,ZRX,UTC+0
1,i,100Tokens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20 days 23:59:00.000000000,2018-10-30 23:59:00,NaN,NaN,NaN,0.00,0.0000,2018-10-10 00:00:00,OHC,UTC+0
2,i,21 Million,-0.082670,NaN,NaN,NaN,0.058150,0.902950,0.038900,NaN,...,6 days 00:00:00.000000000,2017-06-18 00:00:00,NaN,NaN,NaN,0.00,0.0000,2017-06-12 00:00:00,21MCoin,UTC+0
3,i,3D-Token,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55 days 23:59:59.000000000,2018-02-11 23:59:59,NaN,NaN,NaN,0.00,0.0000,2017-12-18 00:00:00,3DT,UTC+0
4,i,8 Circuit Studios – Wave 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30 days 06:59:00.000000000,2018-10-31 23:59:00,NaN,NaN,NaN,0.00,0.0000,2018-10-01 17:00:00,8BT,UTC+0
5,i,AB-CHAIN,-0.015310,NaN,NaN,NaN,0.012600,0.977200,0.010200,NaN,...,40 days 12:00:00.000000000,2018-03-31 21:00:00,NaN,NaN,NaN,48.88,138.6571,2018-02-19 09:00:00,RTB,UTC+0
6,i,ADAMANT Messenger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,151 days 00:01:00.000000000,2018-06-30 06:19:00,NaN,NaN,NaN,0.00,0.0000,2018-01-30 06:18:00,ADM,UTC+3
7,i,AIAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,148 days 23:59:00.000000000,2018-12-05 23:59:00,NaN,NaN,NaN,0.00,0.0000,2018-07-10 00:00:00,EdToken,UTC+2
8,i,AICoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42 days 00:00:00.000000000,2017-08-28 00:00:00,NaN,NaN,NaN,0.05,0.0000,2017-07-17 00:00:00,XAI,UTC+0
9,i,AIOM,0.123475,NaN,NaN,NaN,0.000000,0.962250,0.037750,NaN,...,28 days 00:00:00.000000000,2018-03-15 12:00:00,NaN,NaN,NaN,0.00,0.0000,2018-02-15 12:00:00,AIOM,UTC+2


In [61]:
class ico():
	def __init__(self):
		self.hpages = 3
		self.url = "https://icobench.com/icos?filterSort=rating-desc&page="
		self.s = requests.session()
		self.book = []
		self.r = None

	def scrape(self):
		print("Scraping ICO data...")
		for i in range(1,self.hpages+1):
            headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
            request=Request(self.s.get(self.url+str(i)), headers=headers) 
            response =urlopen(request)
            soup = BeautifulSoup(response, 'html.parser')

            

			for i in x.findAll("div",{"class":"rate color5"}):
				current_ico = (i.parent.parent)
				if not "ico_data" in str(current_ico):
					continue
				for z in current_ico.findAll("div"):
					if "Start" in z.text and "End" in z.text:
						pass
					if "Start" in z.text:
						pattern = re.compile("Start:(.*)")
						start_date = re.findall(pattern,z.text)
					if "End" in z.text:
						pattern = re.compile("End:(.*)")
						end_date = re.findall(pattern,z.text)
				for z in current_ico.findAll("a"):
					if "href" in str(z):
						pattern = re.compile('href="(.*)">')
						link = re.findall(pattern,str(z))
				title = current_ico.find("a",{"class":"name"})
				pattern = re.compile("""href="/ico/(.*)">""")
				title = re.findall(pattern,str(title))
				title = title[0].replace('-',' ').title()

				for z in current_ico.findAll("div",{"class":"content"}):
					for p in z.findAll("p"):
						description = p.text
				try:
					sep = "KYC"
					description = description.split(sep,1)[0]
				except Exception as e:
					pass

				rating = float(i.text)

				start_date = start_date[0]
				end_date = end_date[0]
				link = "https://icobench.com" + link[0]
				self.book.append({'title':title,'start':start_date,'end':end_date,'description':description,'link':link,'rating':rating})

		maxi = self.book[0]['rating']
		self.r = {'weekWinner':self.book[0]['title'],'rate':maxi}
		for i in range(0,len(self.book)):
			if self.book[i]['rating'] > maxi:
				maxi = self.book[i]['rating']
				self.r['rate'] = maxi
				self.r['weekWinner'] = self.book[i]['title']
		print("Scraped successfully")


	def syntax(self):
		message = "📈 Meilleure ICO de ces 4 derniers jours: " + self.r['weekWinner'] + " avec un score de: " + str(self.r['rate']) + "/5\n\n"
		cutting = 0
		for i in range(0,len(self.book)):
			cutting += 1
			message += "\nNom: {}\nDescription: {}\nDébut: {}\nFin: {}\nLien: {}\nNote: {}\n----------------\n".format(self.book[i]['title'],self.book[i]['description'],self.book[i]['start'],self.book[i]['end'],self.book[i]['link'],self.book[i]['rating'])
			if (cutting % 3) == 0:
				message += "+++"
		# adapted to ICObench syntax
		return message

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [4]:
ico = ico()


NameError: name 'ico' is not defined

In [6]:
ico.scrape()

NameError: name 'log' is not defined

In [11]:
def get_json(url):
    '''
    Sends HTTP Request to provided url and returns a json (dictionary) object.

    Arguements: 'url' - Requires a full http address including any applicable API keys.
    '''
    request = Request(url, headers={'User-Agent': 'Python'})
    response = urlopen(request)
    raw_data = response.read()
    json_data = json.loads(raw_data)
    return json_data

In [109]:
x = get_json(url)

In [111]:
x['results']

[{'id': 9989,
  'title': 'DACX',
  'slug': 'dacx',
  'type': 'IEO',
  'ticker': 'DACX',
  'rating': '5.0',
  'status': 'Ongoing',
  'category': 'Trading',
  'country': 'United Kingdom',
  'platform': 'Ethereum',
  'profile_url': 'https://www.trackico.io/ico/dacx/',
  'logo_url': 'https://www.trackico.io/media/img/ico/dacx-digital-asset-commodity-exchange/dacx-digital-asset-commodity-exchange-logo.png',
  'short_description': 'Digital Asset & Commodity Exchange leverages artificial intelligence and high performance computing technology, optimized towards providing an effici…',
  'pre_ico_start': None,
  'pre_ico_end': None,
  'ico_start': '2019-10-25T00:00:00Z',
  'ico_end': '2020-01-10T00:00:00Z'},
 {'id': 9988,
  'title': 'Gigajoule',
  'slug': 'gigajoule',
  'type': 'STO',
  'ticker': 'GIGJ',
  'rating': '4.4',
  'status': 'Pre-Sale',
  'category': 'Other',
  'country': 'Malta',
  'platform': 'Ethereum',
  'profile_url': 'https://www.trackico.io/ico/gigajoule/',
  'logo_url': 'https:

In [1]:
git status

SyntaxError: invalid syntax (<ipython-input-1-d93c8dd246e3>, line 1)

743

In [12]:
from urllib.request import Request, urlopen
import json
api_key = "HR2ZZ9PKUC9R8DFL"
def read_json(url):
    request = Request(url)
    response = urlopen(request)
    data = response.read()
    url2 = json.loads(data)
    return url2

In [ ]:
url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={BTC}&market=USD&apikey={api_key}"
btc = read_json(url)
date, close = [],[]

In [13]:
import time

In [36]:
df = pd.DataFrame()
for i in range(len(ico_df)):
    try:
        ticker = ico_df.index[i]
        ohlcv = get_crypto_daily_price(ticker)
        df[ticker] = ohlcv['close']
        for i in progressbar(range(10), f"Query Success for {ticker}!, Preparing data for next call: ", 40):
                time.sleep(0.1)
        
    except:
        print('Unsuccessfull Query :( Setting value to 0')
        df[ticker] = 0
    
df.head()

Query Success for REP!, Preparing data for next call: [########################################] 10/10
Query Success for LSK!, Preparing data for next call: [########################################] 10/10
Query Success for DGD!, Preparing data for next call: [########################################] 10/10
Query Success for WAVES!, Preparing data for next call: [########################################] 10/10
Query Success for STRAT!, Preparing data for next call: [########################################] 10/10
Query Success for XTO!, Preparing data for next call: [########################################] 10/10
Query Success for INCNT!, Preparing data for next call: [########################################] 10/10
Query Success for NEO!, Preparing data for next call: [########################################] 10/10
Query Success for BLOCKPAY!, Preparing data for next call: [########################################] 10/10
Unsuccessfull Query :( Setting value to 0
Unsuccessfull Query 

KeyboardInterrupt: 

In [14]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = df.fillna(0)

In [15]:
def forecast_price(df):
    '''
    Requires df with ticker index
    '''
    fut_df = pd.DataFrame()
    forecast = {}
    df = df.fillna(0)
    for i in range(len(df)):
        try:
            ticker = df.index[i]
            ohlcv = get_crypto_daily_price(ticker)
            fut_df[ticker] = ohlcv['close']
            for i in progressbar(range(10), f"Query Success for {ticker}!, Preparing data for next call: ", 40):
                    time.sleep(0.1)

        except:
            print(f'Unsuccessfull Query for {ticker} :( Setting value to 0')
            fut_df[ticker] = 0

    for i in df.columns:
        
        model = ARIMA(df[i], order=(6,1,2))
        results = model.fit()
        forecast[i] = results.forecast(steps=20)[0]
        fut_df = pd.DataFrame.from_dict(forecast)
        
    return fut_df

In [43]:
fut_df = pd.DataFrame()
forecast = {}
df = df.fillna(0)

In [17]:
df = get_TrackICOAPI()

In [19]:
df.head()

,country,platform,pre_ico_end,pre_ico_start,rating,status,Name,type,End,Start,Duration,pre_Duration
Ticker,,,,,,,,,,,,
NET,Andorra,Ethereum,2019-04-14 23:59:00,2019-02-18 00:00:00,3.5,Closed,NeuralTrade Network,ICO,2019-07-07 00:00:00,2019-04-15 00:00:00,83 days,55 days 23:59:00
TPCT,South Korea,Ethereum,2019-07-08 01:00:00,2019-04-15 01:00:00,4.1,Closed,NUPay,ICO,2019-09-16 01:00:00,2019-07-15 01:00:00,63 days,84 days 00:00:00
CYD,United Kingdom,Ethereum,2019-02-28 00:00:00,2019-02-01 00:00:00,4.6,Closed,CANYUDO,ICO,2019-07-31 00:00:00,2019-05-01 00:00:00,91 days,27 days 00:00:00
WRT,United States of America,Ethereum,2019-04-14 00:00:00,2019-02-14 00:00:00,5.0,Closed,Warrior Token,ICO,2019-07-14 00:00:00,2019-05-15 00:00:00,60 days,59 days 00:00:00
Pahoo,France,Ethereum,2019-06-05 11:00:00,2019-05-25 11:00:00,2.6,Closed,Pahoo,IEO,2019-07-05 11:00:00,2019-06-25 11:00:00,10 days,11 days 00:00:00


In [16]:
ticker = df.index[:3]
ohlcv = get_crypto_daily_price(ticker)
fut_df[ticker] = ohlcv['close']
for i in progressbar(range(10), f"Query Success for {ticker}!, Preparing data for next call: ", 40):
        time.sleep(0.1)

NameError: name 'df' is not defined

In [18]:
x = forecast_price(df)

C:\Users\cscat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  import sys


Unsuccessfull Query for NET :( Setting value to 0
Unsuccessfull Query for TPCT :( Setting value to 0
Unsuccessfull Query for CYD :( Setting value to 0
Unsuccessfull Query for WRT :( Setting value to 0
Unsuccessfull Query for Pahoo :( Setting value to 0
Unsuccessfull Query for ITCO :( Setting value to 0
Unsuccessfull Query for KLK :( Setting value to 0
Unsuccessfull Query for PLG :( Setting value to 0
Unsuccessfull Query for FFM :( Setting value to 0
Unsuccessfull Query for ERE :( Setting value to 0
Unsuccessfull Query for EDOS :( Setting value to 0
Unsuccessfull Query for  :( Setting value to 0
Unsuccessfull Query for DAYTA :( Setting value to 0
Unsuccessfull Query for GGT :( Setting value to 0
Unsuccessfull Query for Viacash :( Setting value to 0
Unsuccessfull Query for BNANA :( Setting value to 0
Unsuccessfull Query for XCT :( Setting value to 0
Unsuccessfull Query for NBX :( Setting value to 0
Unsuccessfull Query for SAIEX :( Setting value to 0
Unsuccessfull Query for CXG :( Setting

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [30]:
ohlcv = get_crypto_daily_price('BTC')

In [39]:
test = forecast_price(ico_df[:5])

C:\Users\cscat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  import sys


Query Success for REP!, Preparing data for next call: [########################################] 10/10
Query Success for LSK!, Preparing data for next call: [########################################] 10/10
Query Success for DGD!, Preparing data for next call: [########################################] 10/10
Query Success for WAVES!, Preparing data for next call: [########################################] 10/10
Query Success for STRAT!, Preparing data for next call: [########################################] 10/10


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [70]:
forecast_price()

In [42]:
model = ARIMA(df['REP'], order=(6,1,2))
results = model.fit()
forecast[i] = results.forecast(steps=20)[0]
fut_df = pd.DataFrame.from_dict(forecast)

C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()


NameError: name 'forecast' is not defined

In [15]:
def get_crypto_daily_price(ticker):
    ticker = ticker.upper()
    api_key = os.getenv("CC_API")
    crypto_df = pd.DataFrame()
    url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={ticker}&tsym=USD&allData=true&api_key={api_key}"
    raw_data = read_json(url)
    df = pd.DataFrame(raw_data['Data']['Data'])
    df['time'] = pd.to_datetime(df['time'],unit='s')
    df.set_index(df['time'], inplace=True)
    df['close'] = df['close'].astype(float)
    df['var'] = df['close'].pct_change()
    return df.dropna()

In [ ]:
        crypto_df[ticker] = df['close']
    
    #
    new_columns = pd.MultiIndex.from_product([ crypto_df.columns, ["close"]  ])
    crypto_df.columns = new_columns

    return crypto_df

In [42]:
x=  get_crypto_daily_price('BTC')

In [22]:
def read_json(self):
    '''
    Sends HTTP Request to provided url and returns a json (dictionary) object.

    Arguements: 'url' - Requires a full http address including any applicable API keys.
    '''
    request = Request(self.url, headers={'User-Agent': 'Python'})
    response = urlopen(request)
    raw_data = response.read()
    json_data = json.loads(raw_data)
    return json_data